In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import re
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input
from keras.layers.merge import Concatenate
from numpy import asarray
from numpy import zeros
import wget
import zipfile
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.utils.vis_utils import plot_model
import pydot
import pydotplus
import graphviz
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
from IPython.display import display
from sklearn import metrics
from datetime import datetime
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers        
import datetime

In [2]:
class initialize_data:
    
    def __init__(self, file_date, file_path, file_tab, file_values, target_col_name, features):
        """pass"""
        self.file_date = file_date
        self.file_path = file_path
        self.file_tab = file_tab
        self.file_values = file_values
        self.target_col_name = target_col_name
        self.features = features

class import_data(initialize_data):
    
    def __init__(self, file_name, initialize_data, pred_for):
        super().__init__(initialize_data.file_date, initialize_data.file_path, initialize_data.file_tab, initialize_data.file_values, initialize_data.target_col_name, initialize_data.features)
        self.file_name = file_name
        self.pred_for = pred_for
    
    def import_file(self):
        """pass"""
        df = pd.read_excel(self.file_path + self.file_name + self.file_date + '.xlsx', sheet_name = self.file_tab)
        
        if self.pred_for.lower() == 'market':
            df[self.target_col_name] = self.file_name
        
        return df
    
class process_data(initialize_data):
    
    def __init__(self, target_labels, initialize_data):
        super().__init__(initialize_data.file_date, initialize_data.file_path, initialize_data.file_tab, initialize_data.file_values, initialize_data.target_col_name, initialize_data.features)
        self.target_labels = target_labels
        
    def one_hot_label(self, df):
        """pass"""
        df_dummies = pd.get_dummies(df[self.target_col_name]) # create onehot labels for target values
        df = pd.concat([df, df_dummies], axis = 1) # column bind dummies with df
        
        return df
    
    def column_reduce(self, df):
        """pass"""
        col_to_keep = self.features + self.target_labels
        df_reduced = df[col_to_keep] # keeping relevant features only
        
        return df_reduced
    
    def preprocess_text(self, text):
        """pass"""
        #convert input to string
        text = str(text)

        # Remove punctuations and numbers
        cleaner_text = re.sub('[^a-zA-Z]', ' ', text)

        # Single character removal
        cleaner_text = re.sub(r"\s+[a-zA-Z]\s+", ' ', cleaner_text)

        # Removing multiple spaces
        cleaner_text = re.sub(r'\s+', ' ', cleaner_text)

        return cleaner_text

    
    def update_text(self, df, feature):
        """pass"""
        X = []
        text = list(df[feature])

        for t in text:
            X.append(process_data.preprocess_text(self, text = t))

        return X
    
    
class visualize_data(initialize_data):
    
    def __init__(self, initialize_data):
        super().__init__(initialize_data.file_date, initialize_data.file_path, initialize_data.file_tab, initialize_data.file_values, initialize_data.target_col_name, initialize_data.features)
    
    def plot_data(self, df_labels):
        """pass"""
        fig_size = plt.rcParams["figure.figsize"]
        fig_size[0] = 6
        fig_size[1] = 5
        plt.rcParams["figure.figsize"] = fig_size

        df_labels.sum(axis=0).plot.bar()
    
class model_data(initialize_data):
    
    def __init__(self, initialize_data, split_ratio, max_tokens, random_state, output_sequence_length, batch_size, embed_dim, activation_function, activation_function_final, loss_function, optimizer, eval_metric, epoch, embedding_path, model_file_name):
        super().__init__(initialize_data.file_date, initialize_data.file_path, initialize_data.file_tab, initialize_data.file_values, initialize_data.target_col_name, initialize_data.features)
        self.split_ratio = split_ratio
        self.max_tokens = max_tokens
        self.random_state = random_state
        self.output_sequence_length = output_sequence_length
        self.batch_size = batch_size
        self.embed_dim = embed_dim
        self.activation_function = activation_function
        self.activation_function_final = activation_function_final
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.eval_metric = eval_metric
        self.epoch = epoch
        self.embedding_path = embedding_path
        self.model_file_name = model_file_name
        
    def train_val_split(self, X, y):
        """pass"""
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.split_ratio, random_state=self.random_state)
        
        return X_train, X_val, y_train, y_val
        
    def modelling(self, X_train, X_val, y_train, y_val, import_embeddings = "yes"):
        """pass"""
        embeddings_index = {}
        
        if import_embeddings.lower() == "yes":
            path_to_glove_file = self.embedding_path
            
            with open(path_to_glove_file, encoding="utf8") as f:
                for line in f:
                    word, coefs = line.split(maxsplit=1)
                    coefs = np.fromstring(coefs, "f", sep=" ")
                    embeddings_index[word] = coefs
    
        vectorizer = TextVectorization(max_tokens=self.max_tokens, output_sequence_length=self.output_sequence_length)
        text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(self.batch_size)
        vectorizer.adapt(text_ds)
        
        voc = vectorizer.get_vocabulary()
        word_index = dict(zip(voc, range(len(voc))))
        
        num_tokens = len(voc) + 2
        embedding_dim = self.embed_dim
        hits = 0
        misses = 0

        # Prepare embedding matrix
        embedding_matrix = np.zeros((num_tokens, embedding_dim))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # Words not found in embedding index will be all-zeros.
                # This includes the representation for "padding" and "OOV"
                embedding_matrix[i] = embedding_vector
                hits += 1
            else:
                misses += 1
        print("Converted %d words (%d misses)" % (hits, misses))

        embedding_layer = Embedding(
            num_tokens,
            embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
            trainable=False,
        )

        int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
        embedded_sequences = embedding_layer(int_sequences_input)
        x = layers.Conv1D(self.batch_size, 5, activation=self.activation_function)(embedded_sequences)
        x = layers.MaxPooling1D(5)(x)
        x = layers.Conv1D(self.batch_size, 5, activation=self.activation_function)(x)
        x = layers.MaxPooling1D(5)(x)
        x = layers.Conv1D(self.batch_size, 5, activation=self.activation_function)(x)
        x = layers.GlobalMaxPooling1D()(x)
        x = layers.Dense(self.batch_size, activation=self.activation_function)(x)
        x = layers.Dropout(0.5)(x)
        preds = layers.Dense(len(target_labels), activation=self.activation_function_final)(x)
        model = tf.keras.Model(int_sequences_input, preds)
        model.summary()

        x_train = vectorizer(np.array([[s] for s in X_train])).numpy()
        x_val = vectorizer(np.array([[s] for s in X_val])).numpy()

        model.compile(
            loss=self.loss_function, optimizer=self.optimizer, metrics=[self.eval_metric]
        )
        model.fit(x_train, y_train, batch_size=self.batch_size, epochs=self.epoch, validation_data=(x_val, y_val))

        # Export the Model
        string_input = tf.keras.Input(shape=(1,), dtype="string")
        x = vectorizer(string_input)
        preds = model(x)
        end_to_end_model = tf.keras.Model(string_input, preds)
        
        return end_to_end_model, model, vectorizer

        
    def save_model(self, model):
        """pass"""
        model.save(self.model_file_name + datetime.datetime.now().strftime("%Y_%m_%d") + '.h5') # save model


    def load_saved_model(self, model_file_date):
        """pass"""
        new_model = tf.keras.models.load_model(self.model_file_name + model_file_date + '.h5') # load model

        # set up loaded model
        string_input = tf.keras.Input(shape=(1,), dtype="string")
        x = vectorizer(string_input)
        preds = new_model(x)
        end_to_end_model_new = tf.keras.Model(string_input, preds)
        
        return end_to_end_model_new

In [18]:
def model_exe_1():
    """pass"""
    # execute initializer
    init1 = initialize_data(file_date, file_path, file_tab, file_values, target_col_name, features)
    
    # import files
    x = pd.DataFrame()

    for file in file_name:
        x = x.append(import_data(file_name = file, initialize_data = init1, pred_for=pred_for).import_file())
        
    pd.options.mode.chained_assignment = None  # default='warn'

    # preprocess the data

    # extract unique target values
    target_labels = list(x[target_col_name].dropna().drop_duplicates())

    # add target labels
    process1 = process_data(target_labels = target_labels, initialize_data = init1)

    # run one hot encoder on target labels
    x2 = process1.one_hot_label(df = x)

    # reduce to filtered columns
    x3 = process1.column_reduce(df = x2)

    # apply regex to clean up features
    cleaned_desc = process1.update_text(df = x3, feature = 'feature_var')
    cleaned_manuf_desc = process1.update_text(df = x3, feature = 'feature_val2')

    x3.loc[:, 'feature_var'] = cleaned_desc # replace with cleaned product desc
    x3.loc[:, 'feature_val2'] = cleaned_manuf_desc # replace with cleaned manufacturer desc

    notnull = x3["feature_var"] != ""
    x4 = x3[notnull] # remove null records
    
    # initialize visualization
    visual1 = visualize_data(initialize_data = init1)
    
    # visualize data
    visual1.plot_data(df_labels = x4[target_labels])
    
    return init1, target_labels, process1, x4, x

In [4]:
def model_exe_2():
    """pass"""
    # execute model
    model1 = model_data(initialize_data=init1, 
                        split_ratio=split_ratio, 
                        max_tokens=max_tokens, 
                        random_state=random_state, 
                        output_sequence_length=output_sequence_length, 
                        batch_size=batch_size, 
                        embed_dim=embed_dim, 
                        activation_function=activation_function, 
                        activation_function_final=activation_function_final, 
                        loss_function=loss_function, 
                        optimizer=optimizer, 
                        eval_metric=eval_metric, 
                        epoch=epoch,
                        embedding_path=embedding_path,
                        model_file_name=model_file_name)
    
    # train validation split
    X_train, X_val, y_train, y_val = model1.train_val_split(X = x4['feature_var'], y = x4[target_labels])

    print(np.shape(X_train))
    print(np.shape(X_val))
    print(np.shape(y_train))
    print(np.shape(y_val))
    
    # run model
    final_model, model, vectorizer = model1.modelling(X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val, import_embeddings = "yes")
    
    return model1, final_model, model, vectorizer